## Two Tower Model 학습 시키기

### 1. Feature 테이블에서 데이터 읽어와서 train/test set으로 split 하기

In [4]:
import mysql.connector as sql

import pandas as pd

host = 'ssm-develop.db.sinsang.market'
port = 3306
username = 'dealicious'
password = 'tlstkd12!@'
database_name = 'dealicious'

db_connection = sql.connect(host=host, database=database_name, user=username, password=password)

retrievals_df = pd.read_sql('SELECT * FROM rec_retrievals', con=db_connection)

retrievals_df.head(3)

/var/folders/fx/b8s33_bs1nx_2nnz6h7w64qm0000gn/T/ipykernel_42782/1062528239.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  retrievals_df = pd.read_sql('SELECT * FROM rec_retrievals', con=db_connection)


,t_dat,customer_id,article_id,price,sales_channel_id,month_sin,month_cos,age,garment_group_name,index_group_name
0,1537401600000000000,0095c9b47fc950788bb709201f024c5338838a27c59c02...,710390001,0.019051,1,-0.866025,-0.5,20.0,Skirts,Divided
1,1537401600000000000,0095c9b47fc950788bb709201f024c5338838a27c59c02...,633130019,0.016932,1,-0.866025,-0.5,20.0,Jersey Fancy,Divided
2,1537401600000000000,0095c9b47fc950788bb709201f024c5338838a27c59c02...,671057002,0.008458,1,-0.866025,-0.5,20.0,Jersey Fancy,Divided


In [6]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(retrievals_df, test_size=0.2, random_state=42)

In [8]:
train_df["article_id"] = train_df["article_id"].astype(str)  # to be removed
test_df["article_id"] = test_df["article_id"].astype(str)  # to be removed

pandas dataframe 을 tensorflow dataset 으로 변환하기

query embedding에 사용할 feature
- `customer_id`: ID of the customer.
- `age`: age of the customer at the time of purchase.
- `month_sin`, `month_cos`: time of year the purchase was made.

candidate embedding에 사용할 feature
- `article_id`: ID of the item.
- `garment_group_name`: type of garment.
- `index_group_name`: menswear/ladieswear etc.

In [9]:
import tensorflow as tf

query_features = ["customer_id", "age", "month_sin", "month_cos"]
candidate_features = ["article_id", "garment_group_name", "index_group_name"]

def df_to_ds(df):
    return tf.data.Dataset.from_tensor_slices({col : df[col] for col in df})

BATCH_SIZE = 2048
train_ds = df_to_ds(train_df).batch(BATCH_SIZE).cache().shuffle(BATCH_SIZE*10)
val_ds = df_to_ds(test_df).batch(BATCH_SIZE).cache()

2024-04-29 20:13:07.628478: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [10]:
user_id_list = train_df["customer_id"].unique().tolist()
item_id_list = train_df["article_id"].unique().tolist()

garment_group_list = train_df["garment_group_name"].unique().tolist()
index_group_list = train_df["index_group_name"].unique().tolist()

print(f"Number of transactions: {len(train_df):,}")
print(f"Number of users: {len(user_id_list):,}")
print(f"Number of items: {len(item_id_list):,}")
print(garment_group_list)

Number of transactions: 14,784
Number of users: 3,845
Number of items: 6,487
['Jersey Fancy', 'Jersey Basic', 'Accessories', 'Knitwear', 'Trousers Denim', 'Socks and Tights', 'Special Offers', 'Under-, Nightwear', 'Blouses', 'Skirts', 'Trousers', 'Unknown', 'Swimwear', 'Shoes', 'Outdoor', 'Dresses/Skirts girls', 'Dressed', 'Dresses Ladies', 'Woven/Jersey/Knitted mix Baby', 'Shirts', 'Shorts']


### 2. Two Tower Model 학습

다음의 두 모델을 two tower 학습 기법을 통해 학습 시킴.

- `query model` : user, transaction feature를 나타내는 query embedding 생성하는 모델
- `candidate model` : item feature를 나타내는 candidate embedding을 생성하는 모델

In [11]:
EMB_DIM = 16

In [ ]:
from models.query_tower import QueryTower

query_model = QueryTower(user_id_list=user_id_list, emb_dim=EMB_DIM)

query_model.normalized_age.adapt(train_ds.map(lambda x : x["age"]))

# Initialize model with inputs.
query_df = train_df[query_features]
query_ds = df_to_ds(query_df).batch(1)
query_model(next(iter(query_ds)))


In [ ]:
from models.item_tower import ItemTower


item_model = ItemTower(item_id_list=item_id_list, garment_group_list=garment_group_list, index_group_list=index_group_list, emb_dim=EMB_DIM)

item_df = train_df[candidate_features]
item_df.drop_duplicates(subset="article_id", inplace=True)
item_ds = df_to_ds(item_df)

In [ ]:
import tensorflow_addons as tfa
from models.two_tower_model import TwoTowerModel

model = TwoTowerModel(query_model, item_model)
optimizer = tfa.optimizers.AdamW(0.001, learning_rate=0.01)
model.compile(optimizer=optimizer)

In [ ]:
model.fit(train_ds, validation_data=val_ds, epochs=5)

In [ ]:
print(model)